# DCGAN


In this notebook will learn about Generative Adversarial Networks by implementing a DCGAN to generate images from noise. 

**Important:** Set the Colab environment to run on GPU

**Notebook created by [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) for the [Postgraduate course in artificial intelligence with deep learning](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) in [UPC School](https://www.talent.upc.edu/ing/) (2020).**



In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets, utils
from PIL import Image
import numpy as np
import math
from IPython.display import display
from tqdm import tqdm
device = torch.device("cuda")

## Hyperparameters

In [ ]:
num_epochs = 300

lr = 0.0002
betas = (0.5, 0.999)

noise_size = 100
batch_size = 128
num_val_samples = 25
num_classes = 10
num_input_channels = 1

## Dataset
Download and prepare dataset


In [ ]:
train_transforms = transforms.Compose(
            [   
                transforms.Resize(32),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (.5,))
            ])
dataset = datasets.MNIST(root='data', train=True, transform=train_transforms, download=True)

## Data Loader
Create a data loader for the MNIST dataset

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# DCGAN

## Networks
First, lets define our simple generator

### Exercise 1

The generator takes random noise as input and gives an image as output. Your exercise is to create the generator model.

It should follow these guidelines:
* The input will be a vector with random noise of size `noise_size`
* You should first apply a fully connected with output size 512\*4\*4 (channels\*height\*width)
* Then you should apply 3 blocks of:
    * TransposedConvolution with kernel size 4, stride 2 and padding 1. For the first 2 blocks, the output channels should be 256 and 128. For the third block, the output channels should be the correct one to generate images of the dataset.
    * BatchNorm2d except for the last block.
    * ReLU activation for the first 2 blocks and Tanh for the third block.

**Hint**: Remember to use reshape where necessary

In [ ]:
class Generator(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
      
        self.fc = ...
   
        self.convt1 = nn.Sequential(
            ...
        )
        self.convt2 = nn.Sequential(
           ...
        )
        self.convt3 = nn.Sequential(
            ...
        )

    def forward(self, x):
        ...
        return x

Similarly lets define a simple discriminator

### Exercise 2

The discriminator takes an image as input and classifies it between Real or Fake (1 or 0). Your exercise is to create the discriminator model.

It should follow these guidelines:
* The input will be an image of size `[num_input_channels, 32, 32]`
* You should apply 3 blocks of:
    * Convolution with kernel size 4, stride 2 and padding 1. The output channels should be 128, 256 and 512.
    * BatchNorm2d except for the first block.
    * LeakyReLU activation (alpha=0.2)
* Then you should first apply a fully connected with input size 512\*4\*4 (channels\*height\*width) and the correct output size and activation for binary classification


**Hint**: Remember to use reshape/flatten where necessary

In [ ]:
class Discriminator(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            ...
        )
        self.conv2 = nn.Sequential(
            ...
        )
        self.conv3 = nn.Sequential(
            ...
        )
        
        self.fc = nn.Sequential(
           ... 
        )

    def forward(self, x):
        ...
        return x

In [ ]:
generator = Generator().to(device)
optimizer_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=betas)

discriminator = Discriminator().to(device)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=betas)

criterion = nn.BCELoss()

def init_weights(m):
    if type(m) in {nn.Conv2d, nn.ConvTranspose2d, nn.Linear}:
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if m.bias != None:
            torch.nn.init.constant_(m.bias, 0.0)
    if type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

generator.apply(init_weights)
discriminator.apply(init_weights);


## Train function

### Exercise 3

Complete the code. Take into account which labels should be used at each step of the training.

In [ ]:
def train_batch(real_samples, generator, discriminator, optimizer_g, optimizer_d):

    generator.train()
    discriminator.train()
    
    current_batch_size = real_samples.shape[0]
    label_real = torch.ones(current_batch_size, 1, device=device)
    label_fake = torch.zeros(current_batch_size, 1, device=device)

    ####################
    # OPTIMIZE GENERATOR
    ####################

    # Reset gradients
    optimizer_g.zero_grad()

    # Generate fake samples
    z = torch.randn(current_batch_size, noise_size, device=device)
    fake_samples = generator(z)
    
    # Evaluate the generated samples with the discriminator
    predictions_g_fake = discriminator(fake_samples)

    # Calculate error with respect to what the generator wants
    loss_g = criterion(...)

    # Backpropagate
    loss_g.backward()
    
    # Update weights
    optimizer_g.step()
    
    ####################
    # OPTIMIZE DISCRIMINATOR
    ####################

    fake_samples = fake_samples.detach()
    # Reset gradients
    optimizer_d.zero_grad()

    # Calculate discriminator prediction for real samples
    predictions_d_real = discriminator(real_samples)

    # Calculate error with respect to what the discriminator wants
    loss_d_real = criterion(...)

    # Calculate discriminator loss for fake samples
    predictions_d_fake = discriminator(fake_samples)

    # Calculate error with respect to what the discriminator wants
    loss_d_fake = criterion(...)
    
    # Total discriminator loss
    loss_d = (loss_d_real + loss_d_fake) / 2
    loss_d.backward()

    optimizer_d.step()

    return loss_g.item(), loss_d.item()


## Evaluation function

In [ ]:
@torch.no_grad()
def evaluate(generator, z_val):
    generator.eval()
    fake_samples = generator(z_val).cpu()
    # select a sample or create grid if img is a batch
    nrows = int(math.sqrt(fake_samples.shape[0]))
    img = utils.make_grid(fake_samples, nrow=nrows)

    # unnormalize
    img = (img*0.5 + 0.5)*255

    # to numpy
    image_numpy = img.numpy().astype(np.uint8)
    image_numpy = np.transpose(image_numpy, (1, 2, 0))
    return Image.fromarray(image_numpy)


## Train loop

In [ ]:
z_val = torch.randn(num_val_samples, noise_size, device=device)

for epoch in range(num_epochs):

    for i, (real_samples, labels) in enumerate(dataloader):
        real_samples = real_samples.to(device)
        loss_g, loss_d = train_batch(real_samples, generator, discriminator, optimizer_g, optimizer_d)

        if i % 100 == 0:
            fake_images = evaluate(generator, z_val)
            display(fake_images)

            # Show current loss
            print(f"epoch: {epoch+1}/{num_epochs} batch: {i+1}/{len(dataloader)} G_loss: {loss_g}, D_loss: {loss_d}")
